In [341]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [342]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [343]:
data_path = "/content/drive/MyDrive/VPN Deprecated/data"
folder_path = "/content/drive/MyDrive/VPN Deprecated/Notebooks"
output_path = "/content/drive/MyDrive/VPN Deprecated/Output"

In [344]:
# Helpers for IKE versions
def add_service_count(counter, port, service_obj):
    proto = service_obj.get('protocol')
    if not proto:
        return

    if proto == 'IKE':
        ike = service_obj.get('ike', {})

        v1 = ike.get('v1') if isinstance(ike, dict) else None
        v2 = ike.get('v2') if isinstance(ike, dict) else None

        has_v1 = isinstance(v1, dict) and v1.get('accepted_proposal') is True
        has_v2 = isinstance(v2, dict)

        if has_v1 or has_v2:
            if has_v1:
                counter[port]['IKEv1'] += 1
            if has_v2:
                counter[port]['IKEv2'] += 1
        else:
            counter[port]['IKEv1'] += 1
    else:
        counter[port][proto] += 1


def add_ip_vpn_protocols(ip_set, service_obj):
    proto = service_obj.get('protocol')
    if not proto:
        return

    if proto == 'IKE':
        ike = service_obj.get('ike', {})

        v1 = ike.get('v1') if isinstance(ike, dict) else None
        v2 = ike.get('v2') if isinstance(ike, dict) else None

        has_v1 = isinstance(v1, dict) and v1.get('accepted_proposal') is True
        has_v2 = isinstance(v2, dict)


        if has_v1:
            ip_set.add('IKEv1')
        if has_v2:
            ip_set.add('IKEv2')
        if not (has_v1 or has_v2):
            ip_set.add('IKEv1')
    else:
        ip_set.add(proto)


In [345]:
APP_NAMES = [
    "germany.vpn",
    "de.mobileconcepts.cyberghost",
    "com.zoogvpn.android",
    "com.wsandroid.suite",
    "com.vpn99",
    "com.surfshark.vpnclient.android",
    "com.nordvpn.android",
    "com.ixolit.ipvanish",
    "com.instabridge.android",
    # "com.goldenfrog.vyprvpn.app",
    "com.gaditek.purevpnics",
    "com.bitdefender.vpn",
    "ch.protonvpn.android",
    "com.browsec.vpn",
]
for app_name in APP_NAMES:
    print("Processing:", app_name)

    filename = f'{data_path}/{app_name}/{app_name}.json'
    ip_file  = f'{data_path}/{app_name}/{app_name}.csv'

    ips = [line.strip().split(',')[0] for line in open(ip_file, 'r')]
    #ips = [ line.strip() for line in open(ip_file, 'r') ]

    data = {}
    with open(filename, 'r') as f:
        for line in f:
            d = json.loads(line)
            data[d['result']['resource']['ip']] = d['result']['resource']

    vpn_protocols = ['l2tp', 'sstp', 'pptp']

    vpns = set()
    ports = defaultdict(int)
    services = defaultdict(lambda: defaultdict(int))
    vpn_services = defaultdict(lambda: defaultdict(int))
    ip_ports = defaultdict(lambda: defaultdict(int))
    ip_vpn_services = defaultdict(set)
    total = {'total': len(data.keys()), 'vpn': 0}


    for ip in data.keys():
        result = data[ip]
        vpn_ports = set()

        if 'services' not in result:
            continue

        for service in result['services']:
            if 'port' not in service:
                continue

            port = service['port']
            service_name = service.get('protocol', '')

            # Keep ip_ports as original protocol label, but change IKE to IKEv1/IKEv2 if present
            if service_name == 'IKE':
              ike = service.get('ike', {})
              v1 = ike.get('v1') if isinstance(ike, dict) else None
              v2 = ike.get('v2') if isinstance(ike, dict) else None

              has_v1 = isinstance(v1, dict) and v1.get('accepted_proposal') is True
              has_v2 = isinstance(v2, dict)

              if has_v1 and has_v2:
                  ip_ports[ip][port] = 'IKEv1,IKEv2'
              elif has_v1:
                  ip_ports[ip][port] = 'IKEv1'
              elif has_v2:
                  ip_ports[ip][port] = 'IKEv2'
              else:
                  ip_ports[ip][port] = 'IKEv2'

            else:
                ip_ports[ip][port] = service_name

            ports[port] += 1
            add_service_count(services, port, service)

            # "VPN" via labels
            is_vpn_labeled = False
            if 'labels' in service:
                for lab in service['labels']:
                    if lab.get('value') == 'VPN':
                        is_vpn_labeled = True
                        break

            if is_vpn_labeled and port not in vpn_ports:
                add_ip_vpn_protocols(ip_vpn_services[ip], service)

                if ip not in vpns:
                    vpns.add(ip)
                    total['vpn'] += 1

                vpn_ports.add(port)
                add_service_count(vpn_services, port, service)

            if service_name and service_name.lower() in vpn_protocols:
                add_ip_vpn_protocols(ip_vpn_services[ip], service)
                add_service_count(vpn_services, port, service)
                if ip not in vpns:
                    vpns.add(ip)
                    total['vpn'] += 1

    json.dump(ports, open(f'{output_path}/{app_name}/port.json', 'w'))
    json.dump(services, open(f'{output_path}/{app_name}/services.json', 'w'))
    json.dump(vpn_services, open(f'{output_path}/{app_name}/vpn_services.json', 'w'))
    json.dump(ip_ports, open(f'{output_path}/{app_name}/ip_ports.json', 'w'))
    json.dump(total, open(f'{output_path}/{app_name}/total.json', 'w'))

    arr = []
    for ip, s in ip_vpn_services.items():
        arr.append([ip, ','.join(sorted(list(s)))])

    df = pd.DataFrame(arr, columns=['IP', 'Protocols'])
    df.to_csv(f'{output_path}/{app_name}/ip_vpn_protocols.csv', index=False)

Processing: germany.vpn
Processing: de.mobileconcepts.cyberghost
Processing: com.zoogvpn.android
Processing: com.wsandroid.suite
Processing: com.vpn99
Processing: com.surfshark.vpnclient.android
Processing: com.nordvpn.android
Processing: com.ixolit.ipvanish
Processing: com.instabridge.android
Processing: com.gaditek.purevpnics
Processing: com.bitdefender.vpn
Processing: ch.protonvpn.android
Processing: com.browsec.vpn
